In [4]:
!pip install -q --upgrade pip
!pip install -q --upgrade transformers datasets[audio] accelerate torchcodec
# !pip install -q flash-attn --no-build-isolation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.1 MB/s eta 0:00:00


In [2]:
# from transformers.utils import is_torch_sdpa_available

# print(is_torch_sdpa_available())



True


```python

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): WhisperDecoder(
      (embed_tokens): Embedding(51865, 768, padding_idx=50257)
      (embed_positions): WhisperPositionalEmbedding(448, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperDecoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): GELUActivation()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (encoder_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
  (proj_out): Linear(in_features=768, out_features=51865, bias=False)
)


```

In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from datasets import Audio, load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [6]:

model_id = "openai/whisper-small"  # whisper-large-v3

# Scale-Product-Attention (SDPA) - can also use flash_attention_2
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True,
    attn_implementation="sdpa")

model.to(device)
# model


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [7]:
# Enable static cache and compile the forward pass
# torch.compile is currently not compatible with the Chunked long-form algorithm or Flash Attention 2
model.generation_config.cache_implementation = "static"
model.generation_config.max_new_tokens = 256
# model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
# model

In [13]:
from transformers import AutoProcessor
model_id = "openai/whisper-small"  # whisper-large-v3

processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [17]:

text = "This is a test sentence."
encoded_input = processor.tokenizer(text)
# print(encoded_input.keys())
# print(processor.tokenizer.batch_decode(encoded_input["input_ids"]))
# # If you want to see all available tokens:
# print(len(processor.tokenizer.get_vocab()))

    encoded_input.keys() -> KeysView({'input_ids': [50258, 50363, 5723, 307, 257, 1500, 8174, 13, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]})

    processor.tokenizer.batch_decode(encoded_input["input_ids"]) -> ['<|startoftranscript|>', '<|notimestamps|>', 'This', ' is', ' a', ' test', ' sentence', '.', '<|endoftext|>']

    processor.tokenizer.get_vocab() -> {'!': 0, '"': 1, '#': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17,... upto 51864]

```python

dataset_2 = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
dataset_2_sample = iter(dataset_2)
sample_2 = next(dataset_2_sample)
sample_2

audio_data = Audio(processor.feature_extractor.sampling_rate)
audio_data = np.array(sample_2["audio"]["array"])

audio, sr = librosa.load(audio_data, sr=sampling_rate)


```



In [5]:
# dataset1 = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# dataset1 = dataset1.cast_column("audio", Audio(processor.feature_extractor.sampling_rate))
# sample = dataset1["audio"][0]

    Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
    })

    dataset["audio"][0] -> {'path': '1272-128104-0000.flac',
    'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
            0.0010376 ]),
    'sampling_rate': 16000}

```python

dataset_2 = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
dataset_2_sample = iter(dataset_2)
sample_2 = next(dataset_2_sample)
sample_2

{'file': '/Users/sanchitgandhi/.cache/huggingface/datasets/downloads/extracted/aad76e6f21870761d7a8b9b34436f6f8db846546c68cb2d9388598d7a164fa4b/dev_clean/1272/128104/1272-128104-0000.flac',
 'audio': {'path': '1272-128104-0000.flac',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
         0.0010376 ]),
  'sampling_rate': 16000},
 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
 'speaker_id': 1272,
 'chapter_id': 128104,
 'id': '1272-128104-0000'}

```

**Using Onset Detection and Segmentation**:

**Onset Detection**: Identify points in the audio where new sounds or events begin using libraries like librosa.

**Segmentation**: Divide the audio into segments based on onset detection, treating each segment as a token.

```python
import librosa

# Onset detection
onsets = librosa.onset.onset_detect(y=audio_array, sr=sampling_rate)

# Segmentation (example: dividing audio into segments based on onsets)
segments = []
start = 0
for onset in onsets:
    segments.append(audio_array[start:onset])
    start = onset

# Each segment in 'segments' can be considered a token

```

```
processor.feature_extractor -> WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}
```

Chunked Long-Form
Whisper has a receptive field of 30-seconds. To transcribe audios longer than this, one of two long-form algorithms are required:

**Sequential**: uses a "sliding window" for buffered inference, transcribing 30-second slices one after the other

**Chunked**: splits long audio files into shorter ones (with a small overlap between segments), transcribes each segment independently, and stitches the resulting transcriptions at the boundaries


**The sequential long-form algorithm should be used in either of the following scenarios**:

  >Transcription accuracy is the most important factor, and speed is less of a consideration

  >You are transcribing batches of long audio files, in which case the latency of sequential is comparable to chunked, while being up to 0.5% WER more accurate


**Conversely, the chunked algorithm should be used when:**

  >Transcription speed is the most important factor

  >You are transcribing a single long audio file



>By default, Transformers uses the sequential algorithm. To enable the chunked algorithm, pass the chunk_length_s parameter to the pipeline. For large-v3, a chunk length of 30-seconds is optimal.

>To activate batching over long audio files, pass the argument batch_size:

In [21]:
inputs = processor(
    sample["array"],
    sampling_rate=sample["sampling_rate"],
    return_tensors="pt",
    truncation=False,
    padding="longest",
    return_attention_mask=True,
)

# Pad to 3000 frames
max_frames = processor.feature_extractor.nb_max_frames
inputs["input_features"] = inputs["input_features"][:, :max_frames]
input_features = inputs.input_features
input_features_padded = torch.nn.functional.pad(
    input_features, (0, max_frames - input_features.shape[-1])
)
inputs["input_features"] = input_features_padded

inputs = inputs.to(device, dtype=torch_dtype)

inputs = inputs.to(device, dtype=torch_dtype)
# inputs["input_features"] : (batch_size,feature_size,nb_max_frames) : torch.Size([1, 80, 3000])

In [23]:
gen_kwargs = {
    "max_new_tokens": 445,  # Reduced to 445 to stay within the limit of 448
    "num_beams": 1,
    "condition_on_prev_tokens": False,
    "compression_ratio_threshold": 1.35,  # zlib compression ratio threshold (in token space)
    "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    "logprob_threshold": -1.0,
    "no_speech_threshold": 0.6,
    "return_timestamps": True,
    # "max_initial_condition_len": 150,
    "pad_token_id": processor.tokenizer.pad_token_id,
    "eos_token_id": processor.tokenizer.eos_token_id,
    "decoder_start_token_id": processor.tokenizer.bos_token_id,
    "language":'en'
}

pred_ids = model.generate(**inputs, **gen_kwargs)



    pred_ids : torch.Size([1, 26]) -> tensor([[50257, 50259, 50359, 50364,  2221,    13,  2326,   388,   391,   307,
              264, 31467,   306,   295,   264, 10775,  9471,   279,    11,   293,
              321,   366,  5404,   281,  2928, 50588, 50588,   702, 23163,     0,
            50638]])

In [26]:
pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True, decode_with_timestamps=False)
# [' Mr. Quilter is the Apostle of the Middle Classes, and we are glad to welcome his Gospel.']

# ASR Using Pipeline

**Warm-up** : In each warm-up step, it processes a copy of the sample with the pipeline, generating a sequence of 256 new tokens.

**Fast Run** : After the warm-up steps, it processes the sample again with the pipeline, this time without specifying the number of new tokens to generate.


**sdpa_kernel(SDPBackend.MATH)** It's the context manager, which ensures that the attention mechanism uses the math backend for its operations. This can help optimize the performance and stability of the model during these initial runs.

In [2]:
import torch
from torch.nn.attention import SDPBackend, sdpa_kernel
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from tqdm import tqdm

model_id = "openai/whisper-small"  # whisper-large-v3

torch.set_float32_matmul_precision("high")
# won't work with cuda
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [6]:
# Scale-Product-Attention (SDPA) - can also use flash_attention_2
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, dtype=torch_dtype, low_cpu_mem_usage=True,
    attn_implementation="sdpa")

model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [7]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    # chunk_length_s=30,
    batch_size=16,  # batch size for inference - will make inference slow
    dtype=torch_dtype,
    device=device,
)

Device set to use cpu


In [8]:
from datasets import Audio, load_dataset

dataset2 = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
dataset2 = dataset2.cast_column("audio", Audio(processor.feature_extractor.sampling_rate))
sample2 = dataset2["audio"][0]

In [9]:
# 2 warmup steps
for _ in tqdm(range(2), desc="Warm-up step"):
    with sdpa_kernel(SDPBackend.MATH):
        result = pipe(sample2,
                      generate_kwargs={"min_new_tokens": 256, "max_new_tokens": 256})

# fast run
with sdpa_kernel(SDPBackend.MATH):
    result = pipe(sample2)

# result : {'text': ' Mr. Quilter is the Apostle of the Middle Classes, and we are glad to welcome his Gospel.'}

Warm-up step:   0%|          | 0/2 [00:00<?, ?it/s]`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
Warm-up step: 100%|██████████| 2/2 [04:27<00:00, 133.59s/it]


### Get Whisper Tokens

**How it works**: These tokenizers have been trained on large datasets and can efficiently **convert audio into discrete tokens that capture phonetic or acoustic information**.

**Note** : TypeError: The current model class (WhisperModel) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [12]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import librosa

# Load the processor and model
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Load and preprocess the audio
audio_path = '/content/10s_joke.mp3'
audio, sr = librosa.load(audio_path, sr=16000)

# Preprocess the audio to match Whisper input requirements
input_features = processor(audio, return_tensors="pt",sampling_rate=16000).input_features
# input_features.shape = torch.Size([1, 80, 3000])

# Pass the preprocessed audio features to the model and generate the logits
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Convert the predicted IDs (tokens) back to text using the processor
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True,language='en')
tokens = predicted_ids  # These are the token IDs predicted by the model

# print("Transcription:", transcription)
# print("Tokens:", tokens)
# print("Predicted Tokens",processor.tokenizer.batch_decode(tokens))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



    Transcription: [' Hey buddy, how was school? Everybody was mocking me. Everybody was mocking you? Everybody was mocking you?']
    Tokens: tensor([[50258, 50259, 50359, 50363,  1911, 10340,    11,   577,   390,  1395,
                30,  7646,   390, 49792,   385,    13,  7646,   390, 49792,   291,
                30,  7646,   390, 49792,   291,    30]])
    Predicted Tokens ['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Hey buddy, how was school? Everybody was mocking me. Everybody was mocking you? Everybody was mocking you?']

## Get Direct Encoder Output

In [14]:
from transformers import WhisperProcessor, WhisperModel,WhisperForConditionalGeneration
import torch
import librosa

# Load the processor and model
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperModel.from_pretrained("openai/whisper-small")

# Load and preprocess the audio
audio_path = '/content/10s_joke.mp3'
audio, sr = librosa.load(audio_path, sr=16000)

# Preprocess the audio to match Whisper input requirements
input_features = processor(audio, return_tensors="pt",sampling_rate=16000).input_features
# input_features.shape [batch_size,feature_size,nb_max_frames] : torch.Size([1, 80, 3000])
# Pass through the model to get hidden states (embeddings)
with torch.no_grad():
    # Directly access the encoder using model.encoder
    encoder_outputs = model.encoder(input_features)

# Extract the embeddings from the last hidden layer
embeddings = encoder_outputs.last_hidden_state # Shape: (batch_size, sequence_length, hidden_size)

# Optionally, get a single embedding by averaging over the time dimension
audio_embedding = embeddings.mean(dim=1)  # Shape: (batch_size, hidden_size)
# embeddings.shape [batch_size, sequence_length, hidden_size] : torch.Size([1, 1500, 768])
# audio_embedding.shape [batch_size, hidden_size] : torch.Size([1, 768])

### Get Decoder output

In [ ]:
from transformers import WhisperTokenizer, WhisperForConditionalGeneration

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Generate token IDs for decoder input
input_ids = torch.tensor([[tokenizer.bos_token_id]])  # # BOS token ID, shape: (1, 1)

# Instead of calling model.decoder, call model directly with both inputs
# **Change:** Remove input_ids and use decoder_input_ids instead
with torch.no_grad():
    # **Add output_hidden_states=True to get decoder hidden states**
    decoder_outputs = model(decoder_input_ids=input_ids,
                            encoder_outputs=encoder_outputs,
                            output_hidden_states=True) # Use 'encoder_outputs'

# Access the last hidden state from the decoder output
# decoder_outputs is a Seq2SeqLMOutput object, its attributes depend on the model and inputs
# **Change:** decoder_outputs.last_hidden_state to decoder_outputs.decoder_hidden_states[-1]
# to get the last hidden state of the decoder.
last_hidden_state = decoder_outputs.decoder_hidden_states[-1]
# decoder_outputs.keys() : odict_keys(['logits', 'past_key_values', 'decoder_hidden_states', 'encoder_last_hidden_state'])
# last_hidden_state.shape (batch_size, sequence_length, hidden_size) : torch.Size([1, 1, 768])

last_token_hidden_state = last_hidden_state[:, -1, :]
# last_token_hidden_state.shape : torch.Size([1, 768])

# Access logits from decoder_outputs using .logits
logits = decoder_outputs.logits[:, -1, :]  # shape: (batch_size, vocab_size)
probabilities = torch.softmax(logits, dim=-1)  # shape: (batch_size, vocab_size)
predicted_token_id = torch.argmax(probabilities, dim=-1) # shape: (batch_size)
# predicted_token_id : tensor([50259])
# processor.tokenizer.batch_decode(predicted_token_id) : ['<|en|>']

In [ ]:
# --- Accessing and Decoding decoder_hidden_states ---
# Get all decoder hidden states
# all_decoder_hidden_states = decoder_outputs.decoder_hidden_states  # List of tensors

# Convert each hidden state to token IDs (argmax over vocab) and then decode to text
for i, hidden_state in enumerate(all_decoder_hidden_states):
    # Reshape to (batch_size * sequence_length, hidden_size) for argmax
    hidden_state_reshaped = hidden_state.reshape(-1, hidden_state.shape[-1])

    # Get predicted token IDs by taking argmax over vocabulary dimension
    predicted_token_ids = torch.argmax(hidden_state_reshaped, dim=-1)

    # Convert predicted_token_ids to a list to pass to batch_decode
    predicted_token_ids = predicted_token_ids.tolist() # Change here

    # Decode token IDs to text using the tokenizer
    decoded_tokens = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

    print(f"Decoder Hidden State {i}: {decoded_tokens}")


# --- Accessing and Decoding encoder_last_hidden_state ---
encoder_last_hidden_state = encoder_outputs.last_hidden_state  # shape: (batch_size, sequence_length, hidden_size)

# Reshape to (batch_size * sequence_length, hidden_size) for argmax
encoder_hidden_state_reshaped = encoder_last_hidden_state.reshape(-1, encoder_last_hidden_state.shape[-1])

# Get predicted token IDs by taking argmax over vocabulary dimension
predicted_token_ids_encoder = torch.argmax(encoder_hidden_state_reshaped, dim=-1)

# Convert predicted_token_ids_encoder to a list before decoding
predicted_token_ids_encoder = predicted_token_ids_encoder.tolist()  # Change here

# Decode token IDs to text using the tokenizer
decoded_tokens_encoder = tokenizer.batch_decode(predicted_token_ids_encoder, skip_special_tokens=True)



In [ ]:
print(f"Encoder Last Hidden State: {decoded_tokens_encoder}")